In [85]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [86]:
import pandas as pd
df = pd.read_csv(r"C:/Users/admin/Desktop/Project2/Walmart-Sales-Data-Analysis--SQL-Project-main/Walmart Sales Data.csv.csv")

In [87]:
print(df.head())

    Invoice ID Branch       City Customer type  Gender  \
0  750-67-8428      A     Yangon        Member  Female   
1  226-31-3081      C  Naypyitaw        Normal  Female   
2  631-41-3108      A     Yangon        Normal    Male   
3  123-19-1176      A     Yangon        Member    Male   
4  373-73-7910      A     Yangon        Normal    Male   

             Product line  Unit price  Quantity   Tax 5%     Total  \
0       Health and beauty       74.69         7  26.1415  548.9715   
1  Electronic accessories       15.28         5   3.8200   80.2200   
2      Home and lifestyle       46.33         7  16.2155  340.5255   
3       Health and beauty       58.22         8  23.2880  489.0480   
4       Sports and travel       86.31         7  30.2085  634.3785   

         Date      Time      Payment    cogs  gross margin percentage  \
0  2019-01-05  13:08:00      Ewallet  522.83                 4.761905   
1  2019-03-08  10:29:00         Cash   76.40                 4.761905   
2  2019-03-

In [88]:
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time

In [89]:
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Weekday'] = df['Date'].dt.day_name()
df['Hour'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.hour

In [90]:
print(df.isnull().sum())

Invoice ID                 0
Branch                     0
City                       0
Customer type              0
Gender                     0
Product line               0
Unit price                 0
Quantity                   0
Tax 5%                     0
Total                      0
Date                       0
Time                       0
Payment                    0
cogs                       0
gross margin percentage    0
gross income               0
Rating                     0
Day                        0
Month                      0
Year                       0
Weekday                    0
Hour                       0
dtype: int64


In [91]:
print(df.describe(include='all'))

         Invoice ID Branch    City Customer type  Gender         Product line  \
count          1000   1000    1000          1000    1000                 1000   
unique         1000      3       3             2       2                    6   
top     849-09-3807      A  Yangon        Member  Female  Fashion accessories   
freq              1    340     340           501     501                  178   
mean            NaN    NaN     NaN           NaN     NaN                  NaN   
min             NaN    NaN     NaN           NaN     NaN                  NaN   
25%             NaN    NaN     NaN           NaN     NaN                  NaN   
50%             NaN    NaN     NaN           NaN     NaN                  NaN   
75%             NaN    NaN     NaN           NaN     NaN                  NaN   
max             NaN    NaN     NaN           NaN     NaN                  NaN   
std             NaN    NaN     NaN           NaN     NaN                  NaN   

         Unit price     Qua

In [92]:
sales_by_product = df.groupby("Product line")["Total"].sum().sort_values(ascending=False)
print("Total Sales by Product Line:\n", sales_by_product)

Total Sales by Product Line:
 Product line
Food and beverages        56144.8440
Sports and travel         55122.8265
Electronic accessories    54337.5315
Fashion accessories       54305.8950
Home and lifestyle        53861.9130
Health and beauty         49193.7390
Name: Total, dtype: float64


In [93]:
monthly_sales = df.groupby("Month")["Total"].sum()
print("Monthly Sales:\n", monthly_sales)

Monthly Sales:
 Month
1    116291.868
2     97219.374
3    109455.507
Name: Total, dtype: float64


In [94]:
avg_rating = df.groupby("Product line")["Rating"].mean().sort_values(ascending=False)
print("Average Rating by Product Line:\n", avg_rating)

Average Rating by Product Line:
 Product line
Food and beverages        7.113218
Fashion accessories       7.029213
Health and beauty         7.003289
Electronic accessories    6.924706
Sports and travel         6.916265
Home and lifestyle        6.837500
Name: Rating, dtype: float64


In [95]:
gender_payment = df.groupby(["Gender", "Payment"]).size().unstack()
print("Gender-wise Payment Preferences:\n", gender_payment)

Gender-wise Payment Preferences:
 Payment  Cash  Credit card  Ewallet
Gender                             
Female    178          163      160
Male      166          148      185


In [96]:
hourly_traffic = df['Hour'].value_counts().sort_index()
print("Customer Transactions by Hour:\n", hourly_traffic)

Customer Transactions by Hour:
 Hour
10    101
11     90
12     89
13    103
14     83
15    102
16     77
17     74
18     93
19    113
20     75
Name: count, dtype: int64


In [97]:
city_sales = df.groupby("City")["Total"].sum().sort_values(ascending=False).head(5)
print("Top 5 Cities by Total Sales:\n", city_sales)

Top 5 Cities by Total Sales:
 City
Naypyitaw    110568.7065
Yangon       106200.3705
Mandalay     106197.6720
Name: Total, dtype: float64


In [98]:
import sqlite3
conn = sqlite3.connect("walmart_sales.db") 
df.to_sql("sales", conn, if_exists="replace", index=False)

1000

In [99]:
query1 = """
SELECT City, SUM(Total) AS Total_Sales
FROM sales
GROUP BY City
ORDER BY Total_Sales DESC
LIMIT 5;
"""
result1 = pd.read_sql(query1, conn)
print(result1)

        City  Total_Sales
0  Naypyitaw  110568.7065
1     Yangon  106200.3705
2   Mandalay  106197.6720


In [100]:
query2 = """
SELECT [Product line], ROUND(AVG(Rating), 2) AS Avg_Rating
FROM sales
GROUP BY [Product line]
ORDER BY Avg_Rating DESC;
"""
result2 = pd.read_sql(query2, conn)
print(result2)

             Product line  Avg_Rating
0      Food and beverages        7.11
1     Fashion accessories        7.03
2       Health and beauty        7.00
3       Sports and travel        6.92
4  Electronic accessories        6.92
5      Home and lifestyle        6.84


In [101]:
query3 = """
SELECT Payment, COUNT(*) AS Count
FROM sales
GROUP BY Payment
ORDER BY Count DESC;
"""
result3 = pd.read_sql(query3, conn)
print(result3)

       Payment  Count
0      Ewallet    345
1         Cash    344
2  Credit card    311


In [102]:
query4 = """
SELECT Month, ROUND(SUM(Total), 2) AS Monthly_Sales
FROM sales
GROUP BY Month
ORDER BY Month;
"""
result4 = pd.read_sql(query4, conn)
print(result4)

   Month  Monthly_Sales
0      1      116291.87
1      2       97219.37
2      3      109455.51


In [103]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [104]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv(r"C:/Users/admin/Desktop/Project2/Walmart-Sales-Data-Analysis--SQL-Project-main/Walmart Sales Data.csv.csv")

df['Hour'] = pd.to_datetime(df['Time'], format="%H:%M", errors='coerce').dt.hour

df['Month'] = pd.to_datetime(df['Date'], format="%m/%d/%Y", errors='coerce').dt.month

X = df[["Gender", "City", "Product line", "Payment", "Total", "Hour", "Month"]].copy()

le = LabelEncoder()
for col in ["Gender", "City", "Product line", "Payment"]:
    X[col] = le.fit_transform(X[col])

y = df["Customer type"]  

df.to_csv(r"C:/Users/admin/Desktop/Project2/Walmart-Sales-Data-Analysis--SQL-Project-main/cleaned_walmart_data.csv", index=False)

print("Data prepared successfully!")

Data prepared successfully!


In [105]:
print(df.columns)

Index(['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date',
       'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income',
       'Rating', 'Hour', 'Month'],
      dtype='object')


In [106]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [107]:
import numpy as np
import pandas as pd

print("X_train ke missing values:")
print(pd.DataFrame(X_train).isnull().sum())

print("X_test ke missing values:")
print(pd.DataFrame(X_test).isnull().sum())

X_train ke missing values:
Gender            0
City              0
Product line      0
Payment           0
Total             0
Hour            700
Month           700
dtype: int64
X_test ke missing values:
Gender            0
City              0
Product line      0
Payment           0
Total             0
Hour            300
Month           300
dtype: int64


In [108]:
X_train = pd.DataFrame(X_train).dropna()
X_test = pd.DataFrame(X_test).dropna()

In [121]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

X_train = pd.DataFrame(X_train).apply(pd.to_numeric, errors='coerce')
X_test = pd.DataFrame(X_test).apply(pd.to_numeric, errors='coerce')

imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [122]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [123]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.5233333333333333

Classification Report:
               precision    recall  f1-score   support

      Member       0.52      0.52      0.52       149
      Normal       0.53      0.52      0.52       151

    accuracy                           0.52       300
   macro avg       0.52      0.52      0.52       300
weighted avg       0.52      0.52      0.52       300


Confusion Matrix:
 [[78 71]
 [72 79]]


In [124]:
import pandas as pd
df = pd.read_csv(r"C:/Users/admin/Desktop/Project2/Walmart-Sales-Data-Analysis--SQL-Project-main/Walmart Sales Data.csv.csv")

In [125]:
df.to_csv(r"C:/Users/admin/Desktop/Project2/Walmart-Sales-Data-Analysis--SQL-Project-main/cleaned_walmart_data.csv", index=False)